In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# Exercises
1. Attendance Data
Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

You should end up with something like this:

- name
- Billy    0.5250
- Jane     0.6875
- John     0.9125
- Sally    0.7625
- Name: grade, dtype: float64


In [5]:
df = pd.read_csv('attendance.csv')
df.head()

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [8]:
df = df.rename(columns={'Unnamed: 0': 'name'})

In [34]:
# first, melt the columns into a single column called day and put each day's value in a column called attendance
df_melt = df.melt(id_vars=['name'],var_name='day', value_name='attendance')
df_melt.head()

,name,day,attendance
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T


In [35]:
# next make a column with the appropriate numeric value for attendance
df_melt['att_numeric'] = df_melt.attendance.map({'P': 1.0, 'A':0.0, 'T':0.9, 'H':0.5})
df_melt.head()

,name,day,attendance,att_numeric
0,Sally,2018-01-01,P,1.0
1,Jane,2018-01-01,A,0.0
2,Billy,2018-01-01,A,0.0
3,John,2018-01-01,P,1.0
4,Sally,2018-01-02,T,0.9


In [36]:
# now if I groupby the students' names, sum their numeric values and divide by the number of days, wala
att_percent_df = df_melt.groupby('name').att_numeric.sum() / len(df_melt.groupby('day'))
att_percent_df = pd.DataFrame(att_percent_df)
att_percent_df.columns = ['attendance_percentage']
att_percent_df

,attendance_percentage
name,
Billy,0.5250
Jane,0.6875
John,0.9125
Sally,0.7625


## 2. Coffee Levels

* Read the coffee_levels table.
* Transform the data so that each carafe is in it's own column.
* Is this the best shape for the data?

In [69]:
df = pd.read_csv('coffee_levels.csv')
df.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


In [70]:
df.coffee_carafe.value_counts()

x    10
y    10
z    10
Name: coffee_carafe, dtype: int64

In [71]:
# use pivot table to make the new index to be hour, make the columns the unique values from coffee_carafe,
# and make the values for those columns come from coffee_amount
df_pivot = df.pivot(index=['hour'], columns=['coffee_carafe'], values=['coffee_amount'])
df_pivot

coffee_amount                    
coffee_carafe             x         y         z
hour                                           
8                  0.816164  0.189297  0.999264
9                  0.451018  0.521502  0.915990
10                 0.843279  0.023163  0.144928
11                 0.335533  0.235529  0.311495
12                 0.898291  0.017009  0.771947
13                 0.310711  0.997464  0.398520
14                 0.507288  0.058361  0.864464
15                 0.215043  0.144644  0.436364
16                 0.183891  0.544676  0.280621
17                 0.391560  0.594126  0.436677

In [59]:
# a multilevel index for columns looks like it would become cumbersone eventually
# I found that you could droplevel and rename the columns to get rid of it entirely:

# df_pivot.columns = df_pivot.columns.droplevel(0)
# df_pivot.columns = ['x', 'y', 'z']

In [72]:
# However, if each column is supposed to be a variable and each row is an observation, 
# then this df needs to be transposed which could be done with pivot??  or .T
df_pivot.T

hour                               8         9         10        11        12  \
              coffee_carafe                                                     
coffee_amount x              0.816164  0.451018  0.843279  0.335533  0.898291   
              y              0.189297  0.521502  0.023163  0.235529  0.017009   
              z              0.999264  0.915990  0.144928  0.311495  0.771947   

hour                               13        14        15        16        17  
              coffee_carafe                                                    
coffee_amount x              0.310711  0.507288  0.215043  0.183891  0.391560  
              y              0.997464  0.058361  0.144644  0.544676  0.594126  
              z              0.398520  0.864464  0.436364  0.280621  0.436677

In [73]:
df_pivot = df_pivot.T

In [74]:
df_pivot

hour                               8         9         10        11        12  \
              coffee_carafe                                                     
coffee_amount x              0.816164  0.451018  0.843279  0.335533  0.898291   
              y              0.189297  0.521502  0.023163  0.235529  0.017009   
              z              0.999264  0.915990  0.144928  0.311495  0.771947   

hour                               13        14        15        16        17  
              coffee_carafe                                                    
coffee_amount x              0.310711  0.507288  0.215043  0.183891  0.391560  
              y              0.997464  0.058361  0.144644  0.544676  0.594126  
              z              0.398520  0.864464  0.436364  0.280621  0.436677

# Cake Recipes

* Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.
* Tidy the data as necessary.
* Which recipe, on average, is the best?
* Which oven temperature, on average, produces the best results?
* Which combination of recipe, rack position, and temperature gives the best result?



In [81]:
df = pd.read_csv('cake_recipes.csv')
df.head()

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084


In [82]:
df[['recipe', 'position']] = df['recipe:position'].str.split(':', expand=True)
df.head()

,recipe:position,225,250,275,300,recipe,position
0,a:bottom,61.738655,53.912627,74.414730,98.786784,a,bottom
1,a:top,51.709751,52.009735,68.576858,50.228470,a,top
2,b:bottom,57.095320,61.904369,61.196980,99.248541,b,bottom
3,b:top,82.455004,95.224151,98.594881,58.169349,b,top
4,c:bottom,96.470207,52.001358,92.893227,65.473084,c,bottom


In [83]:
df.drop(columns=['recipe:position'], inplace=True)
df.head()

,225,250,275,300,recipe,position
0,61.738655,53.912627,74.414730,98.786784,a,bottom
1,51.709751,52.009735,68.576858,50.228470,a,top
2,57.095320,61.904369,61.196980,99.248541,b,bottom
3,82.455004,95.224151,98.594881,58.169349,b,top
4,96.470207,52.001358,92.893227,65.473084,c,bottom


In [85]:
# I want each row to be one recipe, so I need to get bottom and top expanded for each temp
# BUT maybe first, I need to melt and then reset index??
df_melt = df.melt(id_vars=['recipe', 'position'])


In [87]:
df_melt.pivot_table(index=['recipe', 'position'], columns='variable', values='value')

variable               225        250        275        300
recipe position                                            
a      bottom    61.738655  53.912627  74.414730  98.786784
       top       51.709751  52.009735  68.576858  50.228470
b      bottom    57.095320  61.904369  61.196980  99.248541
       top       82.455004  95.224151  98.594881  58.169349
c      bottom    96.470207  52.001358  92.893227  65.473084
       top       71.306308  82.795477  92.098049  53.960273
d      bottom    52.799753  58.670419  51.747686  56.183110
       top       96.873178  76.101363  59.571620  50.971626

In [90]:
df_tidy = df_melt.reset_index(drop=True)
df_tidy.head()

,recipe,position,variable,value
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207


In [98]:
df_tidy.columns = ['recipe', 'position', 'temp', 'score']

In [99]:
df_tidy.groupby('recipe').score.mean().sort_values(ascending=False)
# b is best on average

recipe
b    76.736074
c    75.874748
a    63.922201
d    62.864844
Name: score, dtype: float64

In [100]:
df_tidy.groupby('temp').score.mean().sort_values(ascending=False)
# 275 is best on average

temp
275    74.886754
225    71.306022
300    66.627655
250    66.577437
Name: score, dtype: float64

In [101]:
df_tidy.head()

,recipe,position,temp,score
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207


In [106]:
df_tidy.sort_values(by='score', ascending=False)

,recipe,position,temp,score
26,b,bottom,300,99.248541
24,a,bottom,300,98.786784
19,b,top,275,98.594881
7,d,top,225,96.873178
4,c,bottom,225,96.470207
11,b,top,250,95.224151
20,c,bottom,275,92.893227
21,c,top,275,92.098049
13,c,top,250,82.795477
3,b,top,225,82.455004


In [ ]:
# this was my first attempt. not sure it's worse or better

# I want each row to be one recipe, so I need to get bottom and top expanded for each temp
df_pivot = df.pivot(index='recipe', columns='position', values=['225', '250', '275', '300'])
df_pivot

225                   250                   275             \
position     bottom        top     bottom        top     bottom        top   
recipe                                                                       
a         61.738655  51.709751  53.912627  52.009735  74.414730  68.576858   
b         57.095320  82.455004  61.904369  95.224151  61.196980  98.594881   
c         96.470207  71.306308  52.001358  82.795477  92.893227  92.098049   
d         52.799753  96.873178  58.670419  76.101363  51.747686  59.571620   

                300             
position     bottom        top  
recipe                          
a         98.786784  50.228470  
b         99.248541  58.169349  
c         65.473084  53.960273  
d         56.183110  50.971626